<a href="https://colab.research.google.com/github/dvschultz/stylegan2-training/blob/main/TWiML_SG2_ADA_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TWiML StyleGAN2-ADA Workshop

[**Slides accompanying this workshop**](https://docs.google.com/presentation/d/1Nv1KQW2a3KMohUDwiNb5XQnR0_nlZ_mXzLdMriklJYg/edit?usp=sharing)

This notebooks was made by [Derrick Schultz](https://twitter.com/dvsch) and [Lia Coleman](https://twitter.com/Lialialiacole), and contains sample code and ideas from [Mikael Christensen](https://colab.research.google.com/drive/1ShgW6wohEFQtqs_znMna3dzrcVoABKIH) and [Dan Shiffman](https://www.youtube.com/watch?v=vEetoBuHj8g).

Derrick Schultz: [Website](https://artificial-images.com/) • [Twitter](https://twitter.com/dvsch) • [Instagram](https://www.instagram.com/dvsmethid/)



## Colab Basics

Colab is a tool from Google that’s similar to Jupyter notebooks. We’ll be using this because we can get a high-powered GPU for free.

Colab is divided into two types of cells: text cells and code cells. To run a code cell you can either press the play button next to the cell or click in the cell and press `Shift` + `Return`.

Let’s run the cell below:

In [ ]:
%tensorflow_version 1.x
!nvidia-smi

To make demoing easier, we can also add some functions to show images in Colab. Run the next cell to enable that code. This will let us display images and videos directly in Colab.

In [ ]:
from IPython.display import Image, display, HTML
import base64
import io

def show_local_mp4_video(file_name, width=640, height=480):
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))
def make_img_grid(images,width=360):
    html = []
    for image in images:
        with open(image, "rb") as img_file:
            my_string = base64.b64encode(img_file.read())
            img_uri = "data:image/png;base64," + my_string.decode('utf8')
        html.append('<img src="{}" style="width:{}px;display:inline;margin:1px"/>'.format(img_uri,str(width)))
    return ''.join(html)
    

## Intall the StyleGAN2-ADA repo 

**Only run the next cell once per session.**


In [ ]:
!git clone https://github.com/dvschultz/stylegan2-ada
!pip install opensimplex

In [ ]:
%cd /content/stylegan2-ada

### Download a trained model from Google Drive
For this workshop we’ll be using NVIDIA’s MetFaces, a model trained on painted portraits from The Met.

In [ ]:
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl -O /content/network.pkl

There are now many trained models available. I recommend checking out [this github repo](https://github.com/justinpinkney/awesome-pretrained-stylegan2) for available models.

Files can be downloaded and dragged into the Files interface to upload to Colab.

## Generating Images

Also known as "Inference", "Evaluation" or "Testing" the model.


In [ ]:
!python generate.py --help

##Options
`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`: This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--trunc`: This sets the truncation amount.

`--outdir`:  Where to save the results.

In [ ]:
!python generate.py generate-images --network="/content/network.pkl" --seeds=0-25 --outdir="./out/" --trunc=0.5

Let’s look at a handful of images. 

In [ ]:
listOfImageNames = ['/content/stylegan2-ada/out/seed0000.png',
                    '/content/stylegan2-ada/out/seed0001.png',
                    '/content/stylegan2-ada/out/seed0010.png',
                    '/content/stylegan2-ada/out/seed0020.png',
                    '/content/stylegan2-ada/out/seed0025.png',
                ]

display(HTML(make_img_grid(listOfImageNames, 200)))


Let’s zip the generated files and download them.

In [ ]:
!zip -r generated-out.zip /content/stylegan2-ada/out

Note: The above command will output it to a directory called `out`. If you already have stuff in `out`, make sure to clear it out. To delete a entire folder in Colab:

In [ ]:
!rm -r "./out/"

## Truncation Traversal


Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. Most people choose between 0.5 and 1.0, but technically it's infinite. 

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

###Options 
`--network`: Again, point this to your .pkl file.

`--seed`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)

In [ ]:
!python generate.py truncation-traversal --network="/content/network.pkl" --seed=0 --start=-2.0 --stop=2.0 --increment=0.1 --outdir="./tt" --fps=30

In [ ]:
show_local_mp4_video("/content/stylegan2-ada/tt/truncation-traversal-seed1-start-2.0-stop2.0.mp4", width=512, height=512)

There should now be a video called something like: `truncation-traversal-seed1-start-2.0-stop2.0.mp4` in the `tt` folder. You can download just the video by right clicking on it.

## Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at two different examples of interpolation: a linear interpolation and a random noise loop.

Both methods require the following options:

`--network`

`--walk-type`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value


### Linear Interpolation in Z Space

Linear interpolation generates a linear path from one seed to another. The makers of StyleGAN say that doing this in the w space produces the best disentangled interpolations. But let’s start by looking at it in z space.

`--seeds`: Use images you generated to control the interpolation points. If your first and last seed are the same this will produce a loop (nice for Instagram and gifs!)

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="line-z" --seeds=0,2,5,10,,20,0 --outdir="./z-walk" --frames 1440

In [ ]:
show_local_mp4_video("/content/stylegan2-ada/z-walk/walk-z-line0-2-5-0-24fps.mp4", width=512, height=512)

### Linear Interpolation in W Space

Next let’s look at linear interpolation in w space. To do this we set `--walk-type` to `line-w`.

I recommend using the exact same seeds so you see the difference. It’s often very subtle but it is different.


In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="line-w" --seeds=0,2,5,0 --outdir="./w-walk"

In [ ]:
show_local_mp4_video("/content/stylegan2-ada/w-walk/walk-w-line0-2-5-0-24fps.mp4", width=512, height=512)

### Noise Loop Interpolation

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--start_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="noiseloop" --start_seed=0 --outdir="./noise1" --diameter=2.0

In [ ]:
show_local_mp4_video("/content/stylegan2-ada/noise1/walk-z-noiseloop-seed0-24fps.mp4", width=512, height=512)

## Style Mixing

Since seeds are just points in a vector space, we can do math things to them, like adding them together. You could do this thru linear interpolation and using the middle frame, but if you want to visualize a number of options here’s a simple script to do it. This takes a number of seeds to produce a grid showing what happens when you add the row and column together (this will make more sense after running it).

In [ ]:
!python style_mixing.py --outdir="./stylemix" --rows=85,100,75,458,1500 --cols=55,821,1789,293 --network="/content/network.pkl"

## Projection

Projection is the process of taking an image from outside the model and finding its nearest representation inside the model. 

First, lets make some folders to upload images to.

In [ ]:
!mkdir uploads; mkdir aligned

Next, upload an image to the `uploads` folder. I recommend finding a portrait at a decent resolution (~1500px square)

Then we need to make sure the photo we uploaded is aligned similarly to the dataset. Run the next cell to create an aligned image.

In [ ]:
!python align_faces.py ./uploads ./aligned

Now we’re ready to process the projection.

### Options 
`--target`: upload an image that you want to project into the model, and then fill the `--target` option with the path to it. This can be any image, as long as it has the same aspect ratio as the dataset that the network was trained on.

`--network`: your model. Just for this example, we're using the FFHQ network in the `--network` option because it's really easy to see what projection is doing when its projection on faces. But of course, you can input your own model as well!


In [ ]:
!python projector.py --outdir="./projection" --target=/content/stylegan2-ada/aligned/br_01.png --network="/content/network.pkl"

In [ ]:
show_local_mp4_video("/content/stylegan2-ada/projection/proj.mp4", width=1024, height=512)

## Fun: Flesh Digressions

[Flesh Digressions](https://aydao.ai/artwork/2020/01/17/fleshdigressions.html) is a technique from [aydao](https://twitter.com/aydaogman). This command will output a flesh digressions video called `circular-25-10-2020-09-00-34-PM.mp4`.

In [ ]:
!python aydao_flesh_digressions.py --pkl "/content/network.pkl" --psi=0.6 --radius_small=40.0 --radius_large=300.00

In [ ]:
show_local_mp4_video("./circular-27-10-2020-10-52-03-PM.mp4", width=512, height=512)

## More to make with StyleGAN2

###Training Your Own Models
Lia and I just finished up a class on custom training StyleGAN2-ADA models. The videos will probably be up on [YouTube](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA) in a few weeks.

We have [a notebook](https://github.com/dvschultz/stylegan2-training/blob/main/Stylegan2_ada_Custom_Training.ipynb) and [a tutorial](https://www.youtube.com/watch?v=KxXqCsl-z2k) available now.

###Advanced Image and Video Generation

* [Audio Reaction](https://github.com/dvschultz/stylegan2-training/blob/main/StyleGAN2_ADA_AudioReactive_Pitch.ipynb)
* [GANspace](https://github.com/dvschultz/Make-ML-Art-with-Google-Colab/blob/master/Ganspace_S2DD.ipynb)
* [Network Bending](https://github.com/dvschultz/ml-art-colabs/blob/master/Network_Bending_Static_Images.ipynb)
* [Network Blending](https://colab.research.google.com/drive/1tputbmA9EaXs9HL9iO21g7xN7jz_Xrko?usp=sharing)

###Artificial Images Classes

Recorded videos from previous classes:
*   [StyleGAN2 In-Depth](https://www.youtube.com/playlist?list=PLWuCzxqIpJs-l4OygaHssyydjOu-AWoHv)
*   [Making Machine Learning Art Datasets](https://www.youtube.com/playlist?list=PLWuCzxqIpJs-YLb2Yc3Y6rGJlYxVWFNoi)
*   [Learn to Use Google Colab](https://www.youtube.com/playlist?list=PLWuCzxqIpJs8PN_bY5FON5mFqZ0I9QoSG)

You can also sign up to be notified when Lia and I offer new classes here.




# Now it’s your turn.

Download a pre-trained pkl file and generate your own images and video.